**CURSO**: Sensores Remotos, Departamento de Geociencias y Medio Ambiente, Universidad Nacional de Colombia - sede Medellín <br/>
**Profesor**: Edier Aristizábal (evaristizabalg@unal.edu.co) <br/>
**Créditos**: El contenido de este cuaderno proviene de varias fuentes: [CDSE](https://jupyterhub.dataspace.copernicus.eu/user/edieraristizabal@gmail.com/lab/tree/RTC%3Asamples/sentinelhub/from_browser_to_jupyter.ipynb). Se ha hecho todo lo posible por identificar a los titulares de los derechos de autor de los materiales utilizados en este libro. El autor ofrece disculpas por cualquier omisión involuntaria y agradecería incluir el reconocimiento correspondiente en futuras ediciones.

# Copernicus Data Space Ecosystem (CDSE)

El Copernicus Data Space Ecosystem (CDSE) es un gran laboratorio de ciencias de la Tierra digital y colaborativo (https://dataspace.copernicus.eu). Este laboratorio tiene:

* Un Almacén de Datos Gigante: Donde están todas las materias primas (imágenes satelitales).

* Un Taller de Análisis: Con diferentes herramientas para procesar y analizar esos datos.

* Una Biblioteca/Galería Pública: Donde puedes compartir tus resultados y usar los métodos de otros.

La Estructura del Ecosistema de Datos Copernicus (CDSE) se puede organizar en tres categorías lógicas:

1. **Acceso y Exploración de Datos (El Almacén)**
   
Aquí es donde buscas, visualizas y descargas los datos satelitales en bruto. Es el primer contacto con la información.

**Copernicus Browser**: Es la herramienta más sencilla y visual. Es un mapa interactivo que te permite navegar, encontrar imágenes de los satélites Sentinel (y otros) para tu área de interés, previsualizarlas y descargarlas. Es perfecto para una primera exploración visual.

**APIs de Acceso a Datos (como STAC y OData)**: Son las "puertas traseras" para programadores. Te permiten automatizar la búsqueda y descarga de datos escribiendo scripts (en Python, por ejemplo), en lugar de hacer clic en un mapa. Es mucho más eficiente si sabes exactamente qué datos necesitas para un área y fechas específicas.

2. **Procesamiento y Análisis (El Taller)**

Una vez que tienes los datos, necesitas herramientas para trabajar con ellos. El CDSE te ofrece varios entornos para que no tengas que descargar petabytes de datos a tu ordenador. El procesamiento se hace "en la nube", cerca de los datos.

**JupyterLab**: Este es tu entorno de programación personal en la nube. Como usas Python, R y JavaScript, te sentirás como en casa. Te dan una máquina virtual con todo preinstalado para que puedas escribir y ejecutar tu código directamente sobre los datos del ecosistema. Ideal para desarrollar, probar tus métodos y hacer análisis detallados en áreas de tamaño moderado.

**openEO**: Piensa en openEO como el "motor de procesamiento a gran escala". No es tanto un entorno de programación, sino una API (un lenguaje común) para decirle a la infraestructura de la nube qué hacer. Usas openEO para aplicar análisis complejos (como los que podrías prototipar en JupyterLab) sobre áreas inmensas (un país, un continente) sin preocuparte por la infraestructura. Es la herramienta para pasar de la investigación a la operación a gran escala.

**Data Workspace**: Es una interfaz web más guiada que te permite generar productos bajo demanda seleccionando un algoritmo y configurando sus parámetros. Es menos flexible que JupyterLab u openEO, pero más sencillo si solo necesitas aplicar un procesamiento estándar.

3. **Colaboración y Difusión (La Biblioteca)**

La ciencia avanza compartiendo. Esta parte del ecosistema se centra en la reutilización y la colaboración.

**openEO Algorithm Plaza**: Como ya vimos, es el mercado de algoritmos. Aquí puedes publicar los flujos de trabajo que creaste con openEO para que otros los usen, o puedes tomar algoritmos de otros para aplicarlos a tus propios problemas.

## Request Builder

[Request Builder](https://shapps.dataspace.copernicus.eu/requests-builder/) es una interfaz gráfica en línea para las API de Sentinel Hub. Esta herramienta facilita la creación y depuración de solicitudes a las API, y permite exportar el código resultante en varios lenguajes de programación. En este tutorial, crearemos una solicitud de la [Process API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html) para descargar imágenes ráster del área en Colombia que previamente examinamos en el Browser. Al igual que una [solicitud de Process API en código](https://github.com/eu-cdse/notebook-samples/blob/main/sentinelhub/introduction_to_SH_APIs.ipynb), una solicitud creada con el Request Builder consiste en 5 partes principales:

- `Data Collection`
- `Time Range`
- `Area of Interest`
- `Output`
- `Evalscript`

![True Color Sentinel-2 image of Wildfires near Lac de la Frégate, Quebec, Canada ](https://i.pinimg.com/736x/ed/ea/14/edea14f72e8263558fc1db7f5c3771c1.jpg)

Cada una de estas puede configurarse de manera independiente en la interfaz. Utiliza los siguientes ajustes para el ejemplo :

- `Data Collection`: sentinel-2 l2a
- `Time Range`: Del 01.01.2025 al 30.09.2025
- `Area of Interest`: [-76.048422,5.583184,-74.839341,6.768261]

Haz clic en `Parse` para cargar el área de interés; esta debería mostrarse en la ventana del mapa y acercarse al rectángulo seleccionado.

- `Evalscript`: usa el siguiente evalscript o puedes hacerlo y tomarlo del Copernicus Browser:

```javascript
function setup() {
    return {
        input: ["B02", "B03", "B04", "B08", "B11", "B12", "dataMask"],
        output: [
      		{ id: "default", bands: 4 },
      		{ id: "burnMask", bands: 1, sampleType: "UINT8" },
    	]
      };
}
function evaluatePixel(samples) {
	var NDWI=index(samples.B03, samples.B08); 
	var NDVI=index(samples.B08, samples.B04);
	var INDEX= ((samples.B11 - samples.B12) / (samples.B11 + samples.B12))+(samples.B08);

  	if((INDEX>0.1)||(samples.B02>0.1)||(samples.B11<0.1)||(NDVI>0.3)||(NDWI > 0.1)){
  		return{
            default:[2.5*samples.B04, 2.5*samples.B03, 2.5*samples.B02, samples.dataMask],
			burnMask:[0]
        }
	}
	else {
 	return {
        default: [1, 0, 0, samples.dataMask],
        burnMask: [1]}
	}
}
```

**Output**: el evalscript tiene dos salidas diferentes: default, con 4 bandas (Rojo, Verde, Azul y Transparencia/Data Mask), y burnMask, que devuelve 1 donde el área está quemada y 0 donde no lo está. Debemos configurar la salida para que se cree un ráster a partir de ambas para cada imagen. Como el área de interés es bastante grande y el límite para una sola solicitud es de 2500×2500 píxeles, reduciremos la resolución a 40 metros. Primero, haz clic en la pestaña `resolution`, luego selecciona:

`Res` X in meters: 40

`Res` Y in meters: 40

`Image format`: TIFF

`Identifier`: default

Ahora debes hacer clic en el botón `Add Response` para crear un segundo conjunto de datos de salida. Este debe tener el mismo formato y llevar el nombre correspondiente a la segunda salida del evalscript:

`Image format`: TIFF

`Identifier`: burnMask

Finalmente, debes configurar el lenguaje del `Request Preview` como python-requests en el menú desplegable. Esto generará el código de la solicitud que luego podrás usar también en un Jupyter Notebook.

Si ahora haces clic en `Send`, el sistema te pedirá guardar la solicitud y descargar la respuesta. La respuesta será un archivo .tar que contiene default.tif y burnMask.tif.